### 1. Préparer nos DataFrames pour estimer le CAPM.
 - Vous verrez que les deux premières colonnes de *rendements* sont constituées des rendements du SP500 (notre proxy du rendement de marché $r_m$) et ceux du TBILL 3 mois (notre proxy du rendement sans risque $r_f$).
 - Nous créons ensuite notre vecteur (colonne) de prime de risque du marché: $R_m-r_f$.
 - Nous créons ensuite un index constitué **seulement** des tickers des actions pour pouvoir créer un dataframe *PR* qui contient les primes de risques de toutes les actions du SP500. 

In [1]:
import sys, ipykernel
print(sys.executable)
print(ipykernel.__version__)


c:\Users\User\Documents\evalactif\seance1eval\.venv\Scripts\python.exe
6.30.1


In [3]:
import pandas as pd
import numpy as np  

rendements = pd.read_csv("C:/Users/User/Documents/evalactif/seance1eval/merged_data (1).csv", index_col=0, parse_dates=True)

rendements["MRP"] = rendements["^GSPC"] - rendements["^IRX"] # Calcul du Market Risk Premium    
actions = rendements.columns.difference(["^GSPC", "^IRX", "MRP"])

PR = rendements[actions].subtract(rendements["^IRX"], axis=0) # Prime de risque des actions

### 2. Estimation du CAPM
Nous voulons vérifier la validité empirique du CAPM. La première méthode consiste à estimer le CAPM en incluant la possibilité d’un **alpha de Jensen**. Nous testerons ensuite si chaque actif $i$ dispose d’un $\alpha_i$ significativement différent de zéro.

Le CAPM théorique prédit que le rendement espéré d’un actif $i$ et celui du marché vérifient la relation suivante :
$$
E[r_i] - r_f = \beta_i \, (E[r_m] - r_f)
$$

Dans les données empiriques, nous observons les séries historiques de rendements $r_{i,t}$ et un taux sans risque variant dans le temps $r_{f,t}$ (étant donné que l'on prend les yields du TBill 3 mois). La régression estimée pour chaque actif $i$ s’écrit alors :
$$
r_{i,t} - r_{f,t} = \alpha_i + \beta_i \, (r_{m,t} - r_{f,t}) + \varepsilon_{i,t}, 
\quad t = 1, \dots, T
$$
où $\alpha_i$ mesure la performance anormale (alpha de Jensen) et $\varepsilon_{i,t}$ représente la partie du rendement non expliquée par le modèle (le terme d'erreur).

Dans le code suivant, j'effectuerais l'esimation du CAPM par calcul matriciel. Vous trouverez ici un condensé de l'esimation par les MCO en écriture matricielle. En notant les vecteurs et matrices suivants :
$$
\mathbf{Y}_i =
\begin{bmatrix} r_{i,1} - r_{f,1} \\ r_{i,2} - r_{f,2} \\ \vdots \\ r_{i,T} - r_{f,T} \end{bmatrix}, \quad
\mathbf{X} =
\begin{bmatrix} 1 & r_{m,1} - r_{f,1} \\ 1 & r_{m,2} - r_{f,2} \\ \vdots & \vdots \\ 1 & r_{m,T} - r_{f,T} \end{bmatrix}, \quad
\boldsymbol{\varepsilon}_i =
\begin{bmatrix} \varepsilon_{i,1} \\ \varepsilon_{i,2} \\ \vdots \\ \varepsilon_{i,T} \end{bmatrix},
$$
la régression peut s’écrire sous la forme compacte :
$$
\mathbf{Y}_i = \mathbf{X} \, \boldsymbol{\theta}_i + \boldsymbol{\varepsilon}_i, \quad
\boldsymbol{\theta}_i = 
\begin{bmatrix} \alpha_i \\ \beta_i \end{bmatrix}.
$$

L’estimateur des MCO s’écrit alors :
$$
\hat{\boldsymbol{\theta}}_i = (\mathbf{X}^\top \mathbf{X})^{-1} \mathbf{X}^\top \mathbf{Y}_i,
$$
où $\hat{\alpha}_i$ est le premier élément et $\hat{\beta}_i$ le second. Les résidus sont donnés par :
$$
\hat{\boldsymbol{\varepsilon}}_i = \mathbf{Y}_i - \mathbf{X}\hat{\boldsymbol{\theta}}_i.
$$
La variance des estimateurs $\sigma^2(\hat{\boldsymbol{\theta}}_i)=\sigma^2(\hat{\epsilon}_i)(X^\top X)^{-1}$.

In [ ]:
betas = []
alphas = []
t_stats_alpha = []
t_stats_beta=[]

PRM = rendements["MRP"].values  #PRM est le market risk premium

for action_i in actions:
    Y = PR[action_i].values  
    X = np.column_stack((np.ones(len(Y)), PRM))  # ajout de la constante

    #estimateurs
    theta_i = np.linalg.inv(X.T @ X) @ X.T @ Y 
    alpha_i, beta_i = theta_i #cela permet de décomposer theta_i en alpha et beta
    alphas.append(alpha_i)
    betas.append(beta_i) 
    
    #calcul des statistiques de student
    eps_i = Y - X @ theta_i
    sig2_eps_i = np.sum(eps_i**2) / (len(Y) - X.shape[1]) 
    cov_theta_i = sig2_eps_i * np.linalg.inv(X.T @ X)  

    se_theta_i = np.sqrt(np.diag(cov_theta_i))
    t_alpha_i, t_beta_i = theta_i / se_theta_i
    t_stats_alpha.append(t_alpha_i)
    t_stats_beta.append(t_beta_i)  

# Vous avez une petite procédure qui permet de rajouter une étoile pour la 
# significativité de votre t_stat à un seuil de 5% (vous pourriez la complexifier pour rajouter ***, **, *
# suivant le degré de significativité que vous voudriez avoir)

capm_annual = pd.DataFrame({
    "Alpha": np.array(alphas) * 252,
    "Beta": betas,
    "t_alpha": [f"{t:.2f}*" if abs(t) > 1.96 else f"{t:.2f}" for t in t_stats_alpha],
    "t_beta": [f"{t:.2f}*" if abs(t) > 1.96 else f"{t:.2f}" for t in t_stats_beta],
    "E[ri]-E[rf]": PR.mean()*252  
}, index=actions)

# Top 10 et Bottom 10 des actifs en fonction de leurs alphas
print(capm_annual.sort_values("Alpha", ascending=False).head(10))

print(capm_annual.sort_values("Alpha", ascending=True).head(10))


         Alpha      Beta t_alpha  t_beta  E[ri]-E[rf]
AMD   0.469682  1.652673   2.56*  26.69*     0.652055
NVDA  0.397151  1.732144   3.10*  39.97*     0.588293
TSLA  0.342559  1.468220    1.88  23.86*     0.504577
AXON  0.311790  1.141289   2.06*  22.30*     0.437731
TPL   0.303899  1.042170   2.06*  20.87*     0.418903
DECK  0.268583  1.120341   2.20*  27.13*     0.392213
BLDR  0.267331  1.570752    1.93  33.56*     0.440663
ANET  0.252195  1.252106   1.97*  28.94*     0.390366
CDNS  0.223515  1.212834   2.78*  44.72*     0.357351
FICO  0.220330  1.309203   2.33*  41.03*     0.364801
         Alpha      Beta t_alpha  t_beta  E[ri]-E[rf]
VTRS -0.226471  0.917725   -1.87  22.38*    -0.125200
PSKY -0.152021  1.156410   -1.02  22.96*    -0.024411
CCL  -0.144365  1.729639   -0.82  28.98*     0.046501
NCLH -0.143590  1.789924   -0.75  27.61*     0.053929
MHK  -0.137180  1.232670   -1.13  30.08*    -0.001155
IVZ  -0.136612  1.534717   -1.36  45.37*     0.032745
WBD  -0.133753  1.047434   -

In [10]:
import random
import random

A = random.randrange(10, 200)

if A > 100:
    print(A, "est supérieur à 100")
elif A < 100:
    print(A, "est inférieur à 100")
elif A == 100:
    print(A, "est égal à 100")
else:
    print("Erreur")


123 est supérieur à 100
